In [16]:
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
from textwrap import dedent
from pprint import pprint

In [18]:
from langchain_community.tools import TavilySearchResults

query = "스테이크와 어울리는 와인을 추천해 주세요."

web_search = TavilySearchResults(max_results=2)

search_results = web_search.invoke(query)

for result in search_results:
    print(result)
    print('-'*100)

H
----------------------------------------------------------------------------------------------------
T
----------------------------------------------------------------------------------------------------
T
----------------------------------------------------------------------------------------------------
P
----------------------------------------------------------------------------------------------------
E
----------------------------------------------------------------------------------------------------
r
----------------------------------------------------------------------------------------------------
r
----------------------------------------------------------------------------------------------------
o
----------------------------------------------------------------------------------------------------
r
----------------------------------------------------------------------------------------------------
(
-----------------------------------------------------------------------

In [19]:
print(type(web_search))

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>


In [20]:
print(web_search.name)

tavily_search_results_json


In [21]:
print(web_search.description)

A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.


In [22]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')

llm_with_tools = llm.bind_tools(tools=[web_search])

In [23]:
query = "안녕하세요."
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print('-'*100)

pprint(ai_msg.content)
print('-'*100)

pprint(ai_msg.tool_calls)
print('-'*100)

AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 82, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'stop', 'logprobs': None}, id='run--64a44afa-cfac-4810-a9a7-478e6d9f206e-0', usage_metadata={'input_tokens': 82, 'output_tokens': 11, 'total_tokens': 93})
----------------------------------------------------------------------------------------------------
'안녕하세요! 어떻게 도와드릴까요?'
----------------------------------------------------------------------------------------------------
[]
----------------------------------------------------------------------------------------------------


In [24]:
query = "스테이크와 어울리는 와인을 추천해 주세요."
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print('-'*100)

pprint(ai_msg.content)
print('-'*100)

pprint(ai_msg.tool_calls)
print('-'*100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_lQBJNQLo7NoIrGBgUMjIqXqf', 'function': {'arguments': '{"query":"스테이크 와인 추천"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 92, 'total_tokens': 115, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--e71deaef-725b-45b7-bada-10f558aa31c0-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '스테이크 와인 추천'}, 'id': 'call_lQBJNQLo7NoIrGBgUMjIqXqf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 92, 'output_tokens': 23, 'total_tokens': 115})
-------------------------------------------------------------

In [25]:
tool_call = ai_msg.tool_calls[0]
tool_call

{'name': 'tavily_search_results_json',
 'args': {'query': '스테이크 와인 추천'},
 'id': 'call_lQBJNQLo7NoIrGBgUMjIqXqf',
 'type': 'tool_call'}

In [26]:
tool_message = web_search.invoke(tool_call)

print(tool_message)

content="HTTPError('401 Client Error: Unauthorized for url: https://api.tavily.com/search')" name='tavily_search_results_json' tool_call_id='call_lQBJNQLo7NoIrGBgUMjIqXqf' artifact={}


In [27]:
pprint(tool_message.content)

("HTTPError('401 Client Error: Unauthorized for url: "
 "https://api.tavily.com/search')")


In [28]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain

today = datetime.today().strftime("%Y-%m-%d")

prompt = ChatPromptTemplate([
    ("system", f"You are helpful AI assistant. Today's date is {today}."),
    ('human', "{user_input}"),
    ("placeholder", "{message}"),
])

llm = ChatOpenAI(model='gpt-4o-mini')

llm_with_tools = llm.bind_tools(tools=[web_search])

llm_chain = prompt | llm_with_tools

@chain
def web_search_chain(user_input: str):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_)

    tool_msgs = web_search.batch(ai_msg.tool_calls)
    print("tool_mags: \n", tool_msgs)
    print('-'*100)

    return llm_chain.invoke({**input_, "message": [ai_msg, *tool_msgs]})

response = web_search_chain.invoke("오늘 모엣샹동 샴페인의 원화 가격은 얼마인가요?")

pprint(response.content)




tool_mags: 
 [ToolMessage(content="HTTPError('401 Client Error: Unauthorized for url: https://api.tavily.com/search')", name='tavily_search_results_json', tool_call_id='call_WMlUgo9VnAoCYXfvRV91dVUi', artifact={})]
----------------------------------------------------------------------------------------------------
''


In [29]:
# 도구 정의하기
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool
from typing import List

@tool
def search_web(query: str) -> str:
    """Searches the internet for information that does not exist in the database or
    for the latest information."""

    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)

    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
        ])

    if len(formatted_docs) > 0:
        return formatted_docs

    return "관련 정보를 찾을 수 없습니다."



In [30]:
query = "스테이크와 어울리는 와인을 추천해 주세요."
search_result = search_web.invoke(query)

print(search_result)

TypeError: string indices must be integers, not 'str'